In [1]:
import pandas as pd
import numpy as np

from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_parquet('../../../metrics.parquet')

In [3]:
df, _ = train_test_split(df, train_size=20000, stratify=df['time_to_stop_activity'], random_state=11)

In [4]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [5]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [6]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [7]:
def train_evaluate_krr(alpha, kernel, gamma, degree, coef0, X_train, Y_train, X_test, Y_test):
    mse = None
    mae = None
    r2 = None

    try:
        krr = KernelRidge(alpha=alpha, kernel=kernel, gamma=gamma, degree=degree, coef0=coef0)
        krr.fit(X_train, Y_train)
        y_pred = krr.predict(X_test)
        
        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)
        
        print(f'KRR - Alpha: {alpha}, Kernel: {kernel}, Gamma: {gamma}, Degree: {degree}, Coef0: {coef0} Finalized - MSE: {mse}, MAE: {mae}, R2: {r2}')

        return {
            'alpha': alpha,
            'kernel': kernel,
            'gamma': gamma,
            'degree': degree,
            'coef0': coef0,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }
    
    except Exception as e:
        print(f'KRR - Alpha: {alpha}, Kernel: {kernel}, Gamma: {gamma}, Degree: {degree}, Coef0: {coef0} Error: {e}')
    
        return {
            'alpha': alpha,
            'kernel': kernel,
            'gamma': gamma,
            'degree': degree,
            'coef0': coef0,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }

In [8]:
param_combinations = [
    (alpha, kernel, gamma, degree, coef0)
    for alpha in [0.1, 1.0, 10.0]
    for kernel in ['linear']
    for gamma in [None, 0.1, 1.0]
    for degree in [3, 4]
    for coef0 in [0.0, 1.0]
]

In [9]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [11]:
results = Parallel(n_jobs=3)(
    delayed(train_evaluate_krr)(alpha, kernel, gamma, degree, coef0, X_train, Y_train, X_test, Y_test)
    for alpha, kernel, gamma, degree, coef0 in param_combinations
)

In [12]:
benchmark_df = pd.DataFrame(results)

In [13]:
benchmark_df.to_excel('KernelRidgeRegressorBenchmark.xlsx')